# تنفيذ نظام خبير للحيوانات

مثال من [منهج الذكاء الاصطناعي للمبتدئين](http://github.com/microsoft/ai-for-beginners).

في هذا المثال، سنقوم بتنفيذ نظام بسيط قائم على المعرفة لتحديد حيوان بناءً على بعض الخصائص الفيزيائية. يمكن تمثيل النظام بالشجرة AND-OR التالية (هذا جزء من الشجرة الكاملة، يمكننا بسهولة إضافة المزيد من القواعد):

![](../../../../../../translated_images/ar/AND-OR-Tree.5592d2c70187f283.webp)


## قشرة أنظمة الخبراء الخاصة بنا مع الاستدلال العكسي

لنحاول تعريف لغة بسيطة لتمثيل المعرفة تعتمد على قواعد الإنتاج. سنستخدم فئات بايثون كالكلمات المفتاحية لتعريف القواعد. سيكون هناك 3 أنواع أساسية من الفئات:
* `Ask` تمثل سؤالًا يحتاج إلى طرحه على المستخدم. تحتوي على مجموعة الإجابات الممكنة.
* `If` تمثل قاعدة، وهي مجرد سكر نحوي لتخزين محتوى القاعدة
* `AND`/`OR` هما فئتان لتمثيل فروع AND/OR للشجرة. هما فقط تخزنان قائمة الوسائط بداخلها. لتبسيط الكود، تم تعريف كل الوظائف في الفئة الأصل `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

في نظامنا، تحتوي الذاكرة العاملة على قائمة من **الحقائق** كـ **أزواج سمات-قيمة**. يمكن تعريف قاعدة المعرفة على أنها قاموس واحد كبير يربط الإجراءات (حقائق جديدة يجب إدراجها في الذاكرة العاملة) بالشروط، المعبر عنها بتعابير AND-OR. أيضًا، يمكن `Ask` عن بعض الحقائق.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

لأداء الاستدلال العكسي، سنقوم بتعريف فئة `Knowledgebase`. ستحتوي على:
* `memory` العمل - قاموس يربط السمات بالقيم
* قواعد Knowledgebase بصيغة المعرفة أعلاه

الطريقتان الرئيسيتان هما:
* `get` للحصول على قيمة سمة، مع إجراء الاستدلال إذا لزم الأمر. على سبيل المثال، `get('color')` ستحصل على قيمة خانة اللون (ستطلب إذا لزم الأمر، وتخزن القيمة للاستخدام لاحقًا في الذاكرة العاملة). إذا طلبنا `get('color:blue')`، فسوف تطلب لونًا، ثم تعيد قيمة `y`/`n` اعتمادًا على اللون.
* `eval` تُجري الاستدلال الفعلي، أي تتنقل في شجرة AND/OR، تقييم الأهداف الفرعية، إلخ.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

الآن دعونا نُعرّف قاعدة معرفتنا عن الحيوانات ونُجري الاستشارة. لاحظ أن هذه المكالمة ستطرح عليك أسئلة. يمكنك الإجابة بكتابة `y`/`n` لأسئلة نعم-لا، أو بتحديد رقم (0..N) لأسئلة ذات إجابات متعددة الأطول.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## استخدام Experta للاستدلال الأمامي

في المثال التالي، سنحاول تنفيذ الاستدلال الأمامي باستخدام إحدى مكتبات تمثيل المعرفة، [Experta](https://github.com/nilp0inter/experta). **Experta** هي مكتبة لإنشاء أنظمة الاستدلال الأمامي في بايثون، وهي مصممة لتكون مشابهة للنظام الكلاسيكي القديم [CLIPS](http://www.clipsrules.net/index.html).

كان بإمكاننا أيضًا تنفيذ السلسلة الأمامية بأنفسنا بدون مشاكل كثيرة، لكن التنفيذات البدائية عادةً ليست فعالة جدًا. من أجل مطابقة القواعد بشكل أكثر فعالية، يُستخدم خوارزمية خاصة تسمى [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

سنقوم بتعريف نظامنا كصف فرعي من `KnowledgeEngine`. يتم تعريف كل قاعدة بواسطة دالة منفصلة مع تعليمة `@Rule`، التي تحدد متى يجب أن يتم تطبيق القاعدة. داخل القاعدة، يمكننا إضافة حقائق جديدة باستخدام دالة `declare`، وإضافة هذه الحقائق سيؤدي إلى استدعاء بعض القواعد الأخرى بواسطة محرك الاستدلال الأمامي.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

بمجرد أن نحدد قاعدة المعرفة، نقوم بملء ذاكرتنا العاملة ببعض الحقائق الأولية، ثم نستدعي طريقة `run()` لأداء الاستدلال. يمكنك أن ترى كنتيجة أنه تمت إضافة حقائق مستنتجة جديدة إلى الذاكرة العاملة، بما في ذلك الحقيقة النهائية حول الحيوان (إذا قمنا بإعداد جميع الحقائق الأولية بشكل صحيح).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**تنويه**:
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي والمعتمد. بالنسبة للمعلومات الحساسة أو الهامة، يُنصح بالاستعانة بالترجمة المهنية البشرية. نحن غير مسؤولين عن أي سوء فهم أو تفسير ناتج عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
